In [1]:
import os
import numpy as np
from simulation_module import SimulationConfig, SimulationState, SimulationRunner, SimulationIO, SimulationVisualizer, AnimationMaker
from laser import Laser
import parralel
from time import time
# from parralel_workers import process_single_rolling_average_density_map
start0 = time()

laser650 = laser650 = Laser(650, [1,0], saturation = 10, detuning = -2 * np.pi * 20 / 2, Gamma = 2 * np.pi * 20)

dt_exponent = -3
num_steps_exponent = 6
assert num_steps_exponent < 6, "Probably this will stall the computer"
num_quenches = 50
num_workers = 7
N = 10
gamma = 0.1

filename = f"{N}_{gamma}_positions_1.json"
assert os.path.exists(filename), f"File '{filename}' not found. You need to generate a starting configuration for that value of N, it seems like you don't have one. Try N = 6, gamma = 1 for example."

# SHORT CONFIG
config = SimulationConfig(
    N = N,
    w = 1.0,
    g = gamma,
    m = 137.327,  # mass in atomic units
    T_mK = 0,     # temperature in mK
    dt = 10**dt_exponent,
    num_steps = int(10**num_steps_exponent),
    damping = False,
    damping_parameter = 1.0,
    langevin_temperature = False,
    lasers=[]  # or a list of Laser instances
)

total_steps_per_worker = num_quenches * config.num_steps/num_workers
print(f"Total number of simulated timesteps per worker = {(total_steps_per_worker / (10**6)):.3f} million steps per worker. Expected time = {(total_steps_per_worker * 0.000095954544):.3f} s")

config.save_shortform()

state = SimulationState(config)
state.positions = SimulationIO.load_positions(f"{N}_{config.g}_positions_1.json")
state.initial_positions = state.positions.copy()
state.initialized = True

# Use Eddie's user-specific scratch space
scratch_base = "" #"/exports/eddie/scratch/s2142953"

quench_trajectory_folder = os.path.join(scratch_base, "quench_folder")
animation_folder = os.path.join(scratch_base, "animation_folder")
temperature_folder = os.path.join(scratch_base, "temperature_folder")
animation_output = "animation_output"
rolling_average_output = "rolling_average_output"

# Make sure folders exist
os.makedirs(quench_trajectory_folder, exist_ok=True)
os.makedirs(animation_folder, exist_ok=True)
os.makedirs(temperature_folder, exist_ok=True)
os.makedirs(animation_output, exist_ok = True)

AssertionError: Probably this will stall the computer

In [ ]:
start = time()
parralel.run_quench_series(
    config = config,
    loadfile = f"{N}_{config.g}_positions_1.json",
    output_dir = quench_trajectory_folder,
    g_start = 0.1,
    g_end = 0.6,
    g_step = num_quenches,
    num_workers = num_workers
)    
print(f"Time taken to quench = {time()-start:.3f} s")

In [ ]:
# start = time()
# parralel.generate_rolling_average_density_map_images_from_quench_folder(
#     traj_folder = quench_trajectory_folder,
#     output_dir = rolling_average_output,
#     base_config = config,
#     time_per_image = 5,
#     overlapping_time = 0,
#     full_histogram = True,
#     animate = True,
#     num_workers = num_workers)
# print(f"Time taken to generate rolling images = {time()-start:.3f} s")

In [ ]:
start = time()
parralel.generate_density_map_images_from_quench_folder(quench_trajectory_folder, 
                                                        animation_folder, 
                                                        config, 
                                                        num_workers = num_workers,
                                                       square = False                                                       )
print(f"Time taken to process images = {time()-start:.3f} s")

In [ ]:
import shutil

# List of folder names to remove
folders_to_delete = ["animation_output", "animation_folder", "quench_folder", "temperature_folder"]  

for folder in folders_to_delete:
    if os.path.exists(folder) and os.path.isdir(folder):
        shutil.rmtree(folder)
        print(f"Deleted folder: {folder}")
    else:
        print(f"Folder not found or not a directory: {folder}")


In [ ]:
# start = time()
# AnimationMaker.make_gif_or_mp4_from_images(animation_folder, f"{animation_output}/--output_animation_dt_1e{dt_exponent}_num_1e{num_steps_exponent}_time_{int(10**(dt_exponent+num_steps_exponent))}us.mp4", fps=6, reverse = True)
# print(f"Time taken to make animation = {time()-start:.3f} s")

In [6]:
# start = time()
# parralel.generate_temperature_plots_from_quench_folder(quench_trajectory_folder,temperature_folder,config, grainyness=100,num_workers=num_workers)
# print(f"Time taken to make temperature plots {time()-start:.5f} s")

In [7]:
# start = time()
# AnimationMaker.make_gif_or_mp4_from_images(temperature_folder, f"{animation_output}/--temperature_animation_dt_1e{dt_exponent}_num_1e{num_steps_exponent}_time_{int(10**(dt_exponent+num_steps_exponent))}us.mp4", fps=6, reverse = False)
# print(f"Time taken to make temperature animation = {time()-start:.3f} s")

In [11]:
config.save_shortform()